<a href="https://colab.research.google.com/github/joramirez-analista-datos/portafolio/blob/main/U9conexiona_multiples_fuentes_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 📘 Conexión a **múltiples fuentes de datos** con **pandas**

Este notebook demuestra cómo **cargar**, **integrar** y **validar** datos desde **CSV**, **Excel**, **JSON/API** y **SQLite** con `pandas`.

---

## 🎯 Objetivos
- Cargar datos desde **CSV**, **Excel**, **JSON/API** y **SQLite**.
- Integrar datos (**merge** horizontal y **concat** vertical).
- Normalizar identificadores y tipos; validar **nulos**, **duplicados** y **reglas de negocio**.
- Exportar un dataset consolidado a **CSV**.


## 0) Librerías

In [ ]:

import pandas as pd
import numpy as np
import sqlite3
import requests




## 1) Cargar datos de **múltiples fuentes**
- **CSV (HTTP):** `tips.csv`
- **Excel (HTTP):** clientes de ejemplo
- **JSON/API:** `https://fakestoreapi.com/products`
- **SQLite (local):** tabla `regiones`


In [4]:

# 1.1 CSV (HTTP)
csv_url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/tips.csv"

ventas = pd.read_csv(csv_url)
print("CSV (ventas) ->", ventas.shape)
ventas.head(10)


NameError: name 'pd' is not defined

In [3]:

# 1.2 Excel (HTTP)
excel_url ="ejemplo_fuentes_datos.xlsx"
clientes = pd.read_excel(excel_url,
                         sheet_name="clientes")
print("Excel (clientes) ->", clientes.shape)
clientes.head(10)
clientes.head(5)

NameError: name 'pd' is not defined

In [ ]:

# 1.3 JSON / API
api_url = "https://fakestoreapi.com/products"
#get-consultar
#post-insertar informacion
#put - actualizar  informacion
#del - borrar informacion

productos = pd.DataFrame(requests.get(api_url, timeout=30).json())
print("JSON/API (productos) ->", productos.shape)
productos[["id","title","category","price"]].head(10)


JSON/API (productos) -> (20, 7)


,id,title,category,price
0,1,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",men's clothing,109.95
1,2,Mens Casual Premium Slim Fit T-Shirts,men's clothing,22.30
2,3,Mens Cotton Jacket,men's clothing,55.99
3,4,Mens Casual Slim Fit,men's clothing,15.99
4,5,John Hardy Women's Legends Naga Gold & Silver ...,jewelery,695.00
5,6,Solid Gold Petite Micropave,jewelery,168.00
6,7,White Gold Plated Princess,jewelery,9.99
7,8,Pierced Owl Rose Gold Plated Stainless Steel D...,jewelery,10.99
8,9,WD 2TB Elements Portable External Hard Drive -...,electronics,64.00
9,10,SanDisk SSD PLUS 1TB Internal SSD - SATA III 6...,electronics,109.00


In [ ]:

# 1.4 SQLite (INMEMORY)
con = sqlite3.connect(":memory:")
regiones = pd.DataFrame({"RegionID":[1,2,3,4],"Region":["Norte","Sur","Centro","Occidente"]})
regiones.to_sql("regiones", con, index=False, if_exists="replace")
regiones_db = pd.read_sql("SELECT * FROM regiones", con)
print("SQLite (regiones) ->", regiones_db.shape)
regiones_db


SQLite (regiones) -> (4, 2)


,RegionID,Region
0,1,Norte
1,2,Sur
2,3,Centro
3,4,Occidente


## 2) Normalización y **claves**

In [ ]:
# Preparar clientes
clientes = clientes.copy()
if "Customer Id" in clientes.columns:
    clientes.rename(columns={"Customer Id":"ClienteID"}, inplace=True)
elif "id" in clientes.columns:
    clientes.rename(columns={"id":"ClienteID"}, inplace=True)
clientes["ClienteID"] = clientes["ClienteID"].astype(str)

# Subset + Región aleatoria
rng = np.random.default_rng(123)
clientes["RegionID"] = rng.choice([1,2,3,4], size=len(clientes))
clientes_sample = clientes[["ClienteID","First Name","Last Name","Email","RegionID"]].copy()
clientes_sample.head(3)

,ClienteID,First Name,Last Name,Email,RegionID
0,C1000,Andrés,Ramírez,user0@correo.com,1
1,C1001,Jorge,Díaz,user1@correo.com,3
2,C1002,Carlos,Martínez,user2@correo.com,3


In [ ]:
# Preparar ventas
ventas = ventas.copy()
# Check if 'OrderID' column already exists before inserting
if "OrderID" not in ventas.columns:
    ventas.insert(0, "OrderID", range(1, 1+len(ventas)))

# ProductID desde API
prod_ids = productos["id"].tolist()
ventas["ProductID"] = [prod_ids[i % len(prod_ids)] for i in range(len(ventas))]

# ClienteID asignado desde clientes
cliente_ids = clientes["ClienteID"].tolist()
ventas["ClienteID"] = [cliente_ids[i % len(cliente_ids)] for i in range(len(ventas))]

ventas.head(3)

,OrderID,total_bill,tip,sex,smoker,day,time,size,ProductID,ClienteID
0,1,16.99,1.01,Female,No,Sun,Dinner,2,1,C1000
1,2,10.34,1.66,Male,No,Sun,Dinner,3,2,C1001
2,3,21.01,3.50,Male,No,Sun,Dinner,3,3,C1002


## 3) **Integración horizontal** (JOIN/MERGE)

In [ ]:
# merge ventas + clientes
ventas_clientes = ventas.merge(clientes, on="ClienteID", how="left", validate="m:1")

# Merge clientes + regiones (si se requiere)
clientes_enriq = clientes.merge(regiones_db, on="RegionID", how="left", validate="m:1")

#merge ventas_clientes + productos
ventas_full = ventas_clientes.merge(
    productos[["id","title","category","price"]].rename(columns={"id":"ProductID"}),
    on="ProductID", how="left", validate="m:1"
)

print("Union Fuentes Datos ->", ventas_full.shape)

ventas_full.head(3)

Union Fuentes Datos -> (244, 17)


,OrderID,total_bill,tip,sex,smoker,day,time,size,ProductID,ClienteID,First Name,Last Name,Email,RegionID,title,category,price
0,1,16.99,1.01,Female,No,Sun,Dinner,2,1,C1000,Andrés,Ramírez,user0@correo.com,4,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",men's clothing,109.95
1,2,10.34,1.66,Male,No,Sun,Dinner,3,2,C1001,Jorge,Díaz,user1@correo.com,3,Mens Casual Premium Slim Fit T-Shirts,men's clothing,22.30
2,3,21.01,3.50,Male,No,Sun,Dinner,3,3,C1002,Carlos,Martínez,user2@correo.com,2,Mens Cotton Jacket,men's clothing,55.99


## 4) **Integración vertical** (concat)

In [ ]:
lote_extra = ventas_full.sample(25, random_state=42)
integrado = pd.concat([ventas_full, lote_extra], axis=0, ignore_index=True)
print("Antes:", ventas_full.shape, "| Después:", integrado.shape)
integrado.head(2)

Antes: (244, 17) | Después: (269, 17)


,OrderID,total_bill,tip,sex,smoker,day,time,size,ProductID,ClienteID,First Name,Last Name,Email,RegionID,title,category,price
0,1,16.99,1.01,Female,No,Sun,Dinner,2,1,C1000,Andrés,Ramírez,user0@correo.com,4,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",men's clothing,109.95
1,2,10.34,1.66,Male,No,Sun,Dinner,3,2,C1001,Jorge,Díaz,user1@correo.com,3,Mens Casual Premium Slim Fit T-Shirts,men's clothing,22.30


## 5) **Calidad de datos**

In [ ]:

# Duplicados por OrderID
dups = integrado.duplicated(subset=["OrderID"]).sum()
print("Duplicados por OrderID:", dups)

# Nulos por columna
integrado.isna().sum().sort_values(ascending=False).head(10)


Duplicados por OrderID: 25


,0
Email,15
OrderID,0
total_bill,0
sex,0
tip,0
day,0
time,0
size,0
smoker,0
ProductID,0


In [ ]:

# Reglas simples
rules = {
    "price >= 0": int((integrado["price"] < 0).sum()) if "price" in integrado else 0,
    "total_bill >= 0": int((integrado["total_bill"] < 0).sum()) if "total_bill" in integrado else 0,
    "size >= 1": int((integrado["size"] < 1).sum()) if "size" in integrado else 0,
}
rules


integrado

,OrderID,total_bill,tip,sex,smoker,day,time,size,ProductID,ClienteID,First Name,Last Name,Email,RegionID,title,category,price
0,1,16.99,1.01,Female,No,Sun,Dinner,2,1,C1000,Andrés,Ramírez,user0@correo.com,4,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",men's clothing,109.95
1,2,10.34,1.66,Male,No,Sun,Dinner,3,2,C1001,Jorge,Díaz,user1@correo.com,3,Mens Casual Premium Slim Fit T-Shirts,men's clothing,22.30
2,3,21.01,3.50,Male,No,Sun,Dinner,3,3,C1002,Carlos,Martínez,user2@correo.com,2,Mens Cotton Jacket,men's clothing,55.99
3,4,23.68,3.31,Male,No,Sun,Dinner,2,4,C1003,Andrés,López,user3@correo.com,2,Mens Casual Slim Fit,men's clothing,15.99
4,5,24.59,3.61,Female,No,Sun,Dinner,4,5,C1004,Sofía,Martínez,user4@correo.com,2,John Hardy Women's Legends Naga Gold & Silver ...,jewelery,695.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,34,20.69,2.45,Female,No,Sat,Dinner,4,14,C1033,Carlos,López,user33@correo.com,2,Samsung 49-Inch CHG90 144Hz Curved Gaming Moni...,electronics,999.99
265,20,20.65,3.35,Male,No,Sat,Dinner,3,20,C1019,Lucía,Gómez,user19@correo.com,3,DANVOUY Womens T Shirt Casual Cotton Short,women's clothing,12.99
266,110,14.31,4.00,Female,Yes,Sat,Dinner,2,10,C1009,Sofía,García,user9@correo.com,1,SanDisk SSD PLUS 1TB Internal SSD - SATA III 6...,electronics,109.00
267,31,9.55,1.45,Male,No,Sat,Dinner,2,11,C1030,Sofía,Rodríguez,user30@correo.com,3,Silicon Power 256GB SSD 3D NAND A55 SLC Cache ...,electronics,109.00


In [ ]:

# Limpieza básica
integrado_fix = integrado.drop_duplicates(subset=["OrderID"], keep="first").copy()
if "category" in integrado_fix.columns:
    integrado_fix["category"] = integrado_fix["category"].fillna("unknown")
print("Tamaño final:", integrado_fix.shape)
integrado_fix.head(2)


Tamaño final: (244, 17)


,OrderID,total_bill,tip,sex,smoker,day,time,size,ProductID,ClienteID,First Name,Last Name,Email,RegionID,title,category,price
0,1,16.99,1.01,Female,No,Sun,Dinner,2,1,C1000,Andrés,Ramírez,user0@correo.com,4,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",men's clothing,109.95
1,2,10.34,1.66,Male,No,Sun,Dinner,3,2,C1001,Jorge,Díaz,user1@correo.com,3,Mens Casual Premium Slim Fit T-Shirts,men's clothing,22.30


## 6) **Exportar** CSV consolidado

In [ ]:
# En Colab: guarda en /content; local: carpeta actual
out_csv = "datos_integrados_con_pandas.csv"
try:
    import google.colab  # type: ignore
except Exception:
    out_csv = "datos_integrados_con_pandas.csv"

integrado_fix.to_csv(out_csv, index=False, encoding="utf-8")
out_csv

'datos_integrados_con_pandas.csv'